## Employee Database SQL Example

In this notebook we are going to review some employee databases from a made-up company to demonstrate how we can use SQL to connect databases from different departments or systems.  Have you ever needed information at a company where data is separated into silos by department? Wouldn’t it be nice if you could connect this information across databases to have all the data you need in one place? Well don’t worry! We are going to do just that.

But how?

*	First, we are going to do some data modeling by creating an entity-relationship diagram (ERD) to better visualize this common problem. We are going to be using Quick DBD which can be found [__here__](https://www.quickdatabasediagrams.com/).

*	Then we will do some data engineering with PostgreSQL in order to create our table schema.
*	Lastly, we will use SQLAlchemy, Pandas and Python to show our table data in our notebook.

Below is an ERD of our starting data. As you can see, each table is siloed by itself.

<img src="images/employee_database/erd_start.png">

Let’s say you worked at this company and your boss asked you for a table of the following information:

* employee number, first name, last name, title, department number, department name, department start date and salary. 

Based on the way this data is setup you would have to go to all of the separate sources below and manually add this data together. Thankfully, with SQL, there is a better way. 

<img src="images/employee_database/erd_highlighted.png">

In order to connect these tables, we need to find unique identifiers that each table shares.  Let’s use employee number (emp_no) as our unique identifier since it is a unique value for each employee.  We can make the emp_no in the Employees table our primary key (PK) and the use it as a foreign key (FK) in other tables. This links the tables together on emp_no.

<img src="images/employee_database/erd_pk1.png">

But what about our Departments table? We can do the same as before and use department number (dept_no) as a primary key (PK) and link it to the Dept_Emp table as a foreign key (FK).

<img src="images/employee_database/erd_pk2.png">

Since the Dept_Emp table has both our keys (emp_no and dept_no) it acts as our join table meaning that we can now access all  of our tables in a single database and it will be linked to the same employee number or department number.

Below is what our code looks like in PostgreSQL to do the exact same as we did in our ERD but in a live database.

In [ ]:
### Create table for employees CSV ###
CREATE TABLE employees (
	emp_no INT PRIMARY KEY,
	brith_date DATE,
	first_name VARCHAR(255),
	last_name VARCHAR(255),
	gender VARCHAR (1),
	hire_date DATE
);

### Create table for departments CSV ###
CREATE TABLE departments (
	dept_no VARCHAR(15) PRIMARY KEY,
	dept_name VARCHAR(255)
);

### Create table for titles CSV ###
CREATE TABLE titles (
	emp_no INT NOT NULL,
	FOREIGN KEY (emp_no) REFERENCES employees (emp_no),
	title VARCHAR(255),
	from_date DATE,
	to_date DATE
);

### Create table for salaries CSV ###
CREATE TABLE salaries (
	emp_no INT NOT NULL,
	FOREIGN KEY (emp_no) REFERENCES employees (emp_no),
	salary INT,
	from_date DATE,
	to_date DATE
);

### Create table for dept_emp CSV ##
CREATE TABLE dept_emp (
	emp_no INT NOT NULL,
	FOREIGN KEY (emp_no) REFERENCES employees (emp_no),
	dept_no VARCHAR(15),
	FOREIGN KEY (dept_no) REFERENCES departments (dept_no),
	from_date DATE,
	to_date DATE
);

Now that our table schema is complete, let’s import SQLAlchemy, Pandas and Json. Note that I am only importing Json to conceal my username and password for PostgreSQL.

In [1]:
# Import Dependencies
from sqlalchemy import create_engine  
import pandas as pd
import json

# Pull from json config file with personal SQL postgres information
with open('config.json') as json_data_file:
    data = json.load(json_data_file)

In [2]:
# Define personal pastgres information
username = data["user"]
password = data["passw"]
host = data["host"]
database = data["database"]

In [3]:
# Create Engine
engine = create_engine(f"postgresql://{username}:{password}@{host}/{database}")
conn = engine.connect()

Using SQLAlchemy we can query off of our SQL database and then use Pandas to store our queries as dataframes. Below is our employee table as an example. 

In [4]:
# Create variable and query all records in the employees table
employees = pd.read_sql("SELECT * FROM employees", conn)

# Print first five rows
employees.head()

,emp_no,brith_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


Since all of our tables are connected by primary and foreign keys, we can write a single query to pull in all of the columns our boss requested. Note that we still need to join our tables by emp_no and dept_no. Although we are still joining our tables by a common unique identifier, we can be certain that the salary we pull in from the salaries table will match the title we pull in from the titles table since they are all connected by the unique employee number. This is because we set primary and foreign keys when setting up our schema. 

In [5]:
# Create variable and query all records in the employees table
final_table = pd.read_sql("SELECT e.emp_no, e.first_name, e.last_name, t.title,\
                          de.dept_no, d.dept_name, de.from_date, s.salary\
                          FROM employees e\
                          JOIN titles t ON t.emp_no = e.emp_no \
                          JOIN dept_emp de ON de.emp_no = e.emp_no\
                          JOIN departments d ON d.dept_no = de.dept_no\
                          JOIN salaries s ON s.emp_no = t.emp_no" , conn)

# Print first five rows
final_table.head()

,emp_no,first_name,last_name,title,dept_no,dept_name,from_date,salary
0,10005,Kyoichi,Maliniak,Senior Staff,d003,Human Resources,1989-09-12,78228
1,10005,Kyoichi,Maliniak,Staff,d003,Human Resources,1989-09-12,78228
2,10010,Duangkaew,Piveteau,Engineer,d006,Quality Management,2000-06-26,72488
3,10010,Duangkaew,Piveteau,Engineer,d004,Production,1996-11-24,72488
4,10011,Mary,Sluis,Staff,d009,Customer Service,1990-01-22,42365


For extra points with our new boss we can also filter our table with the WHERE clause. In this example we are only querying the Sales department.

In [6]:
# Create variable and query all records in the employees table
final_table = pd.read_sql("SELECT e.emp_no, e.first_name, e.last_name, t.title,\
                          de.dept_no, d.dept_name, de.from_date, s.salary\
                          FROM employees e\
                          JOIN titles t ON t.emp_no = e.emp_no \
                          JOIN dept_emp de ON de.emp_no = e.emp_no\
                          JOIN departments d ON d.dept_no = de.dept_no\
                          JOIN salaries s ON s.emp_no = t.emp_no\
                          WHERE dept_name = 'Sales'", conn)

# Print first five rows
final_table.head()

,emp_no,first_name,last_name,title,dept_no,dept_name,from_date,salary
0,10002,Bezalel,Simmel,Staff,d007,Sales,1996-08-03,65828
1,10016,Kazuhito,Cappelletti,Staff,d007,Sales,1998-02-11,70889
2,10034,Bader,Swan,Staff,d007,Sales,1995-04-12,47561
3,10041,Uri,Lenart,Senior Staff,d007,Sales,1989-11-12,56893
4,10041,Uri,Lenart,Staff,d007,Sales,1989-11-12,56893


And there you have it, a complex and common problem has been solved with the help of SQL!